In [1]:
import gym
from gym.spaces import Box, Discrete, Tuple, MultiDiscrete
import logging
import random

from ray.rllib.env import MultiAgentEnv
import ray
import os
from ray import tune
from ray.tune import function
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import PPOTrainer, PPOTFPolicy, PPOTorchPolicy
import ray.rllib.agents.ppo as ppo
import numpy as np
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from gym import spaces
from pypylon import pylon
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
import random
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
from torchvision import models, transforms
from torch import nn
import logging

Instructions for updating:
non-resource variables are not supported in the long term
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
logger = logging.getLogger(__name__)

In [3]:
class Camera_Env(gym.Env):
    def __init__(self, camera, 
                       Width = 417,
                       Height = 404):
        
        self.camera = camera
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.Width = Width
        self.Height = Height
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # see https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        print(number_of_actions)
        
        self.action_space = Discrete(number_of_actions)
        # image
        self.observation_space = Box(0, 255, (2048,))#(len(np.zeros((404, 417, 3)).flatten()),))
#         self.observation_space = Box(low=0, 
#                                     high=255, 
#                                     shape=(Height, Width, 3),
#                                     dtype = np.uint8)
        
        print('---------------camera env init-------------------')
        
    def grab_image(self):
        # see https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                break
        self.camera.StopGrabbing()
        return image
    
    def basler(self, action):  
        ExposureTimeRaw = int(self.range_ex_time[action])
        print('ExposureTimeRaw:', ExposureTimeRaw) 
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(5) 
        
    def check_hist(self, img):
        #print('check img')
        hist , bin_edges = np.histogram(np.array(img).ravel())
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin > 50 and max_value_bin < 150:
            pic_ok = True
        else:
            pic_ok = False
        return pic_ok
    
    def reset(self):
        self.camera.ExposureTimeRaw.Value = 20000
    
    def step(self, action):
        self.basler(action)
#         self.lens_environment.lens_movement(46.0)
        img = self.grab_image()
        done = self.check_hist(img)
        if done:
            reward = 1
        else:
            reward = -1
        return img, reward, done, {}
    


In [4]:
class Lens_Env(gym.Env):
    def __init__(self, camera,
                        Height = 404, 
                         Width = 417):
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        self.Width = Width
        self.Height = Height
        
        self.action_space = MultiDiscrete([69, 99])
        # image
        self.observation_space = Box(0, 255, (2048, ))#(len(np.zeros((404, 417, 3)).flatten()),))
#         self.observation_space = Box(low=0, 
#                                             high=255, 
#                                             shape=(self.Height, self.Width, 3),
#                                             dtype = np.uint8)
        self.camera_env = Camera_Env(camera)
        print('---------------lens env init-------------------')
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(2)
        
    def reset(self):
        self.lens_movement(24.0)
        
    def step(self, action):
        self.lens_movement(action)
        img = self.camera_env.grab_image()
        return img, -1, False, {}

In [5]:
class BaslerEnv(MultiAgentEnv):
    def __init__(self, 
                       Width = 417,
                       Height = 404,
                       threshold = 85,
                       filepath = r'C:\Users\CIG\Documents\corning_basler_RL\h_Rl'):
        
        self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.camera.Open()

        self.basler_env = Camera_Env(self.camera)
        self.corning_env = Lens_Env(self.camera)
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        self.threshold = threshold
        self.filepath = filepath
        
        
        self.resnet152 = models.resnet152(pretrained=True)
        modules=list(self.resnet152.children())[:-1]
        self.resnet152=nn.Sequential(*modules)
        for p in self.resnet152.parameters():
            p.requires_grad = False
        
        print(True)
        
    def take_features(self, img):
        preprocess = transforms.Compose([
                            transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        
        image = Image.fromarray(np.uint8(img)).convert('RGB')
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        
        # Run through the convolutional layers and resize the output.
        features_output = self.resnet152(input_batch)
        classifier_input = features_output.view(1, -1)
        
        return classifier_input.cpu().detach().numpy()[0]
        
        
    def reset(self):
        #print('reset')
        self.basler_env.reset()
        self.corning_env.reset()
        
        self.steps_remaining_at_level = None
        self.num_high_level_steps = 0
        
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        obs = self.basler_env.grab_image()
        #print('reset|grab img')
        feature_obs = self.take_features(obs)
        
        return {"high_level_agent": feature_obs,}
    
    def focus_value(self, img):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(img),cv2.CV_64F, 0, 1, ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    
    def step(self, action_dict):
        if "high_level_agent" in action_dict:
            return self._high_level_step(action_dict["high_level_agent"])
        else:
            return self._low_level_step(list(action_dict.values())[0])
        
    
    def _high_level_step(self, action):
        
        self.action_main = action
        
        logger.debug("High level agent action".format(action))
        
        #self.obs_camera, reward_camera, done_c, _ = self.basler_env.step(action)
        self.basler_env.basler(action)
        self.corning_env.lens_movement(46.0)
        obs_camera = self.basler_env.grab_image()
        self.done_camera = self.basler_env.check_hist(obs_camera)
        
        self.steps_remaining_at_level = 10
        self.num_high_level_steps += 1
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        feature_obs = self.take_features(obs_camera)
        obs = {self.low_level_agent_id: feature_obs}
        rew = {self.low_level_agent_id: 0}
        
        done = {"__all__": False}
        
        return obs, rew, done, {}
    
    def _low_level_step(self, action):
        
        action_lens_coarse = 0
        action_lens_fine = action[1]
        if action[0] < 24:
            action_lens_coarse = 24
        else:
            action_lens_coarse = action[0]
        action_lens = float("%s.%s"% (str(action_lens_coarse), str(action_lens_fine)))
        
        logger.debug("Low level agent step {}".format(action_lens))
        done = {"__all__": False}
        if self.done_camera == True:
            self.steps_remaining_at_level -= 1

            obs_lens, reward_lens, done_lens, _ = self.corning_env.step(action_lens)
            feature_obs = self.take_features(obs_lens)
            obs = {self.low_level_agent_id: feature_obs}

            #rew = {self.low_level_agent_id: 1}
            prediction_yolo = self.model_yolo(obs_lens)
            final_result_yolo = self.find_class(prediction_yolo)

 
            flag = False #for save img information
            if final_result_yolo.size == 0:
                rew = {self.low_level_agent_id: -100 + 1*(10 - self.steps_remaining_at_level)}
                if self.steps_remaining_at_level == 0:
                    done[self.low_level_agent_id] = True
                    rew["high_level_agent"] = 0
                    obs["high_level_agent"] = feature_obs
#                 else:
#                     rew = {self.low_level_agent_id: -100 + 1*(10 - self.steps_remaining_at_level)}

            else:
                image = Image.fromarray(obs_lens)
                image = image.crop((final_result_yolo[0], 
                                 final_result_yolo[1], 
                                 final_result_yolo[2], 
                                 final_result_yolo[3]))
                foc_value = self.focus_value(image)
                
                if foc_value > self.threshold:
                    rew = {self.low_level_agent_id: 1}
                    done["__all__"] = True
                    logger.debug("high level final reward {}".format(1))
                    flag = True
                    rew["high_level_agent"] = 1
                    obs["high_level_agent"] = feature_obs
                elif self.steps_remaining_at_level == 0 and foc_value < self.threshold:
                    rew = {self.low_level_agent_id: 0}
                    done[self.low_level_agent_id] = True
                    rew["high_level_agent"] = 0
                    obs["high_level_agent"] = feature_obs
                elif self.steps_remaining_at_level != 0 and foc_value < self.threshold:
                    rew = {self.low_level_agent_id: 1 - 0.001*(foc_value - 85)**2}
                else:
                    print('new case|foc value:',foc_value, 
                          'steps_remaining_at_level:', self.steps_remaining_at_level)
        else:
            feature_obs = self.take_features(self.obs_camera)
            rew = {self.low_level_agent_id: 0}
            obs = {self.low_level_agent_id: feature_obs}
            done[self.low_level_agent_id] = True
            rew["high_level_agent"] = -10
            obs["high_level_agent"] = feature_obs
            
        path = r'C:\Users\CIG\Documents\corning_basler_RL\cringe' + "/" + str(int(self.action_main))
        im = Image.fromarray(obs_lens)
        filename = path + "\img_%s.png" % (str(action_lens))
        im.save(filename)

#         im = Image.fromarray(obs_lens)
#         filename = self.filepath + "\img_%s_%s.png" % (str(action_lens), 
#                                                            str(flag))
#         im.save(filename)
        
        print('action:', action, 'info:',  done, rew)
        return obs, rew, done, {}

In [6]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return  BaslerEnv(env_config)

register_env("BaslerEnv", env_creator)

In [7]:
# Create an instant camera object with the camera device found first.
# camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
# camera.Open()

In [8]:
from ray.rllib.models import ModelCatalog
from ray import tune
from ray.tune import grid_search

In [9]:
from stable_baselines import PPO2

In [10]:
#ModelCatalog.register_custom_model("PPO_2",PPO2)

In [11]:
#len(np.zeros((404, 417, 3)).flatten())

In [12]:
def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"
    
    
stop = {"training_iteration": 200,
        "timesteps_total": 100,
        "episode_reward_mean": 0.0,}



config = {
    "env": BaslerEnv,
    "num_workers": 0,
    "entropy_coeff": 0.01,
    "multiagent": {
        "policies": {
            
            "high_level_policy": (PPOTFPolicy, 
                                  Box(0, 255, (2048,)),#(len(np.zeros((404, 417, 3)).flatten()),)),
                                  Discrete(148), 
                                  {"gamma": 0.9}),
            
            "low_level_policy": (PPOTFPolicy,
                                 Box(0, 255, (2048,)),#(len(np.zeros((404, 417, 3)).flatten()),)),
                                 MultiDiscrete([69, 99]), 
                                 {"gamma": 0.0}),
            
#             "low_level_policy": (PPOTFPolicy,
#                                  Tuple([
#                                      lens_env.observation_space,
#                                      Discrete(148)
#                                  ]), maze.action_space, {
#                                      "gamma": 0.0
#                                  }),
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "framework": "tf",#"torch", #if args.torch else "tf",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
}

#from ray.rllib.agents.trainer_template import build_trainer

# <class 'ray.rllib.agents.trainer_template.MyCustomTrainer'>
# MyTrainer = build_trainer(
#     name="MyCustomTrainer",
#     default_policy=MyTFPolicy)

# ray.init()
# tune.run(MyTrainer
trainer = ppo.PPOTrainer(env= "BaslerEnv", config=config)
# results = tune.run("PPO_2", config=config, stop=stop)

417 404
148
---------------camera env init-------------------
eCOMCaspErr: 0 0
417 404
148
---------------camera env init-------------------
---------------lens env init-------------------


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   

Adding autoShape... 


2021-03-09 20:25:24,245	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!


True
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


From C:\Users\CIG\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2021-03-09 20:25:25,266	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
Trainable.setup took 13.475 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
Install gputil for GPU system monitoring.


In [13]:
for i in range(100):
    # Perform one iteration of training the policy with PPO\n",
    result = trainer.train()
    print(pretty_print(result))
    print('-------', i, '-------')
#     print(evaluation_results)
    if i % 2 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

2021-03-09 20:25:34,956	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


action: [56 29] info: {'__all__': False} {'low_level_1': -99}
action: [51 25] info: {'__all__': False} {'low_level_1': -98}
action: [61 73] info: {'__all__': False} {'low_level_1': -97}
action: [29 71] info: {'__all__': False} {'low_level_1': -96}
action: [48 27] info: {'__all__': True} {'low_level_1': 1, 'high_level_agent': 1}
action: [44 23] info: {'__all__': False} {'low_level_1': -99}
action: [ 6 61] info: {'__all__': False} {'low_level_1': -98}
action: [55 11] info: {'__all__': False} {'low_level_1': -97}
action: [45 60] info: {'__all__': False} {'low_level_1': -96}
action: [ 9 14] info: {'__all__': False} {'low_level_1': -95}
action: [61 67] info: {'__all__': False} {'low_level_1': -94}
action: [43 86] info: {'__all__': False} {'low_level_1': -93}
action: [31 93] info: {'__all__': False} {'low_level_1': -92}
action: [52 42] info: {'__all__': False} {'low_level_1': -91}
action: [36 94] info: {'__all__': False, 'low_level_1': True} {'low_level_1': -90, 'high_level_agent': 0}
action

KeyboardInterrupt: 

In [ ]:
# stop = {
#     "training_iteration": 100,
#     "timesteps_total": 1000,
# }

# results = tune.run("PPO", stop=stop, config=config, checkpoint_freq=5, verbose=1)